In [2]:
from selenium.webdriver import (Chrome, Firefox)
import time
import requests
from bs4 import BeautifulSoup
import pymongo

In [3]:
url = "https://www.leafly.com/hybrid/blue-dream/reviews"
url2 = "https://www.leafly.com/hybrid/blue-dream/reviews?page=2"
sel = "ul#strain-reviews-container"
#sel = 'button#btc-button btc-button--primary'
#<button class="btc-button btc-button--primary" tabindex="0" id="tou-continue">Yes</button>

In [4]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [21]:
docs.count_documents(filter={})

0

In [18]:
docs.drop()

In [7]:
def get_url(x,y):
    """X= number of urls to grab, y= strain name or other thing we are scraping 
    Pull in urls from a 'Load More' button and the like. Create identical URL's with differing numbers"""
    urls = []
    url = 1 
    while url < x:
        urls.append("https://www.leafly.com/%s/reviews?page=%d" % (y, url))
        url += 1 
    return urls

In [49]:
def get_reviews(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    keys{'url', 'html'} and their corresponding values"""
    htmls = []
    for url in l:
        r = requests.get(url)
        html = (r.content)
        htmls.append(html)
    table_check(l)
    results = dict(zip(l, htmls))
    return results

In [47]:
def table_check(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [77]:
def parse_docs(dict):
    """Parse the HTML docs that we have stored in a dictionary, return as a list. """
    strain_text= []
    stars = []
    star_num = []
    for key, values in dict.items():
        soup = BeautifulSoup(values, 'html.parser')
        strain_text.append(soup.select("p.strain-review__text"))
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
            for s in stars:
                star = int((s[6:].split(';')[0]).strip('px'))
                star_num.append(star/22)
                                       
    return dict(zip(strain_text , star_num))

In [63]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [70]:
def star_int_conv(l):
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append(star/22)
    return star_num

In [39]:
blue_dream_reviews = get_url(54, 'hybrid/blue-dream')

## blue_dream_reviews

In [40]:
blue_dream_reviews

['https://www.leafly.com/hybrid/blue-dream/reviews?page=1',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=2',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=3',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=4',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=5',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=6',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=7',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=8',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=9',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=10',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=11',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=12',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=13',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=14',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=15',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=16',
 'https://www.leafly.com/hybrid/b

In [50]:
bdr = get_reviews(blue_dream_reviews)

In [51]:
len(bdr)

53

In [54]:
parse_docs(bdr)

[[<p class="strain-review__text">"Friends, stoners, red-eyed countrymen, lend me your ears; for I bring unto thee a tale of the Blue Dream...
  
  T’was a calm April night, 2014 it was, and I had eagerly purchased an eighth of some pungent Blue Dream. It’s abundance of sugary trichomes, paired with the thick density of the bud was enough to bring a tear to your eye. I enthusiastically ground up the cheeba, packed a generous bowl and went to town. Eight m..."</p>,
  <p class="strain-review__text">"I consider this strain perfect for my psychological problems. almost every other weed with this high sativa content seems to cause extreme paranoia but nothing at all like. i felt completely free and loving and more compasionate about myself than i ever have. i have always had an extremely low self esteem and even though i knew that during the experience my depression was lifted enough for me to easily cope with it. ..."</p>,
  <p class="strain-review__text">"The "Bayer" aspirin of bud that sh

In [65]:
stars = get_stars_list(bdr)

In [71]:
star_int_conv(stars)

[5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 3.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 4.0,
 2.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 2.0,
 4.0,
 5.0,
 5.0,
 5.0,
 2.0,
 0.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 3.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 2.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 3.0,
 5.0,
 5.0,
 5.0,
 5.0

In [44]:
# Strip width stuff so its just an int
int((five_stars[6:].split(';')[0]).strip('px'))

NameError: name 'five_stars' is not defined

In [78]:
parse_docs(bdr)

TypeError: 'dict' object is not callable